### 1. action_new
```python
def action_new(account, action, actor, permission, data) -> int
```

### 2. action_add_permission
```python
def action_add_permission(actor, permission) -> int
```

### 3. transaction_new
```python
def transaction_new(expiration, ref_block_num, ref_block_prefix, \
                    max_net_usage_words, max_cpu_usage_ms, delay_sec) -> int
```


### 4. transaction_send
```python
def transaction_send(trx_id, sender_id, payer, replace_existing) -> None
```

### 5. transaction_add_action
```python
transaction_add_action(trx_id, action_id) -> None
```

### 6. transaction_add_context_free_action
```python
def transaction_add_context_free_action(trx_id, action_id) -> None

```

### 7. transaction_cancel
```python
def transaction_cancel(action_id) -> None

```

#### 创建一个新的transaction:

```python
trx_id = transaction_new(0, 0, 0, 0, 0, 5)
```

transaction_new中的6个参数分别对应libraries/chain/include/eosio/chain/transaction.hpp头文件中的transaction结构体中的成员变量：

```c++
struct transaction_header {
    time_point_sec         expiration;
    uint16_t               ref_block_num       = 0U;
    uint32_t               ref_block_prefix    = 0UL;
    fc::unsigned_int       max_net_usage_words = 0UL;
    uint8_t                max_cpu_usage_ms    = 0;
    fc::unsigned_int       delay_sec           = 0UL;
    ...
}
```


#### 创建一个新的action:
```python
a = action_new(receiver, N('dosomething'), receiver, N('active'), b'hello,worddddd')
```
action_new对应libraries/chain/include/eosio/chain/action.hpp中的数据结构：

```c++
struct action {
  account_name               account;
  action_name                name;
  vector<permission_level>   authorization;
  bytes                      data;
}
```


#### 将action添加到transaction:
```python
transaction_add_action(trx_id, a)
```

#### 发送deferred transaction
```python
sender_id = 1
transaction_send(trx_id, sender_id, payer, 0)
```

#### 取消deferred transaction
```
transaction_cancel(1)
```
transaction_cancel的返回值为1时表示取消成功，为0时表示指定的deferred transaction不存在


In [92]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')


wallet.import_key('test', '5Ji6xuMzbv8UsvJxqFzzknBhWJQZKAjqAKcxnRtMehViNfS1CSS')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='test11111111'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)
print('done!')

done!


In [4]:
eosapi.create_key()

{'public': 'EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL',
 'private': '5Ji6xuMzbv8UsvJxqFzzknBhWJQZKAjqAKcxnRtMehViNfS1CSS'}

In [ ]:
owner_key = 'EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL'
active_key = 'EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL'

eosapi.create_account('test11111111', 'helloworld11', owner_key, active_key, 64*1024, 1.0, 1.0)

In [43]:
eosapi.get_account('helloworld11')

{
    "account_name": "helloworld11",
    "head_block_num": 1161,
    "head_block_time": "2019-06-11T03:47:00.000",
    "privileged": false,
    "last_code_update": "1970-01-01T00:00:00.000",
    "created": "2019-06-11T03:42:46.000",
    "core_liquid_balance": "1000000.0000 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 0,
        "available": "578145772800",
        "max": "578145772800"
    },
    "cpu_limit": {
        "used": 0,
        "available": "110117491200",
        "max": "110117491200"
    },
    "ram_usage": 3558,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL",
                        "weight": 1
                    }
                ],
                "account

In [93]:
account_name = 'helloworld11'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL",
                "weight": 1
            },
        ],
        "accounts": [],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')

done!


In [94]:
account_name = 'helloworld11'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":'helloworld11',"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')


done!


In [72]:
eosapi.get_account('helloworld11')

{
    "account_name": "helloworld11",
    "head_block_num": 16275,
    "head_block_time": "2019-06-11T08:15:06.000",
    "privileged": false,
    "last_code_update": "2019-06-11T06:37:46.000",
    "created": "2019-06-11T03:42:46.000",
    "core_liquid_balance": "1000000.0000 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 7062,
        "available": "181193932792938",
        "max": "181193932800000"
    },
    "cpu_limit": {
        "used": 11514,
        "available": "34559999988486",
        "max": "34560000000000"
    },
    "ram_usage": 5384,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS6DW7yDB68zSD7GbUycAkHk6XAxwxfuHMHPJBuz2YexsdnjSZvL",
                        "weight": 1
                    }
                ],
      

In [81]:
code = '''
import struct
self_account = N('helloworld11')

def apply(receiver, code, action):
    if action == N('dosomething'):
        print(publication_time())
        print(current_time())
        print(now())
#        eosio_assert(0, 'hello,world')
    elif action == N('onerror'):
        print('oops, receive onerror action')
        data = read_action_data()
        sender_id = data[:16]
        sender_id = int.from_bytes(sender_id, 'little')
        trx = data[16:]
        print('+++++++read_action_data:', sender_id, trx)
    elif action == N('senddefer'):
        trx_id = transaction_new(0, 0, 0, 0, 0, 10)

        action_id = action_new(self_account, N('dosomething'), self_account, N('active'), b'hello,worddddd')
        transaction_add_action(trx_id, action_id)

        payer = self_account
        transaction_send(trx_id, 1, payer, False)
    elif action == N('canceldefer'):
        ret = transaction_cancel(1)
        print(ret)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)

True

# 调用senddefer, deferred transaction将廷迟10秒执行

In [82]:
name = 'helloworld11'
r = eosapi.push_action(name, 'senddefer', b'hello,world', {name:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r)


{'transaction_id': '4cf1e8e5c45af113f1497cd6ec5b5b010ccc10b56373699d5aefdf2c0279df40', 'processed': {'id': '4cf1e8e5c45af113f1497cd6ec5b5b010ccc10b56373699d5aefdf2c0279df40', 'block_num': 16736, 'block_time': '2019-06-11T08:18:56.500', 'producer_block_id': None, 'receipt': {'status': 'executed', 'cpu_usage_us': 722, 'net_usage_words': 19}, 'elapsed': 722, 'net_usage': 152, 'scheduled': False, 'action_traces': [{'receipt': {'receiver': 'helloworld11', 'act_digest': '12a9c28dbba0dc36a44e70dda2766ad6024039cb818871fe734cae269b291610', 'global_sequence': 16877, 'recv_sequence': 23, 'auth_sequence': [['helloworld11', 41]], 'code_sequence': 8, 'abi_sequence': 8}, 'act': {'account': 'helloworld11', 'name': 'senddefer', 'authorization': [{'actor': 'helloworld11', 'permission': 'active'}], 'data': '68656c6c6f2c776f726c64'}, 'context_free': False, 'elapsed': 429, 'console': '', 'trx_id': '4cf1e8e5c45af113f1497cd6ec5b5b010ccc10b56373699d5aefdf2c0279df40', 'block_num': 16736, 'block_time': '2019-0

In [62]:
eosapi.get_scheduled_transactions(True, '')

{
    "transactions": [
        {
            "trx_id": "5439ef87cd1e2b039a5761fcf3dccb83e3fef367cddc2612fe0af8d61ee4948b",
            "sender": "helloworld11",
            "sender_id": "0x01000000000000000000000000000000",
            "payer": "helloworld11",
            "delay_until": "2019-06-11T06:37:59.000",
            "expiration": "2019-06-11T06:47:59.000",
            "published": "2019-06-11T06:37:49.000",
            "transaction": {
                "expiration": "2019-06-11T06:37:49",
                "ref_block_num": 5814,
                "ref_block_prefix": 288273723,
                "max_net_usage_words": 0,
                "max_cpu_usage_ms": 0,
                "delay_sec": 10,
                "context_free_actions": [],
                "actions": [
                    {
                        "account": "helloworld11",
                        "name": "dosomething",
                        "authorization": [
                            {
                               

In [63]:
eosapi.get_transaction('5439ef87cd1e2b039a5761fcf3dccb83e3fef367cddc2612fe0af8d61ee4948b')

{
    "id": "5439ef87cd1e2b039a5761fcf3dccb83e3fef367cddc2612fe0af8d61ee4948b",
    "trx": {
        "receipt": {
            "status": "executed",
            "cpu_usage_us": 321,
            "net_usage_words": 0,
            "trx": [
                0,
                "5439ef87cd1e2b039a5761fcf3dccb83e3fef367cddc2612fe0af8d61ee4948b"
            ]
        }
    },
    "block_time": "2019-06-11T06:37:59.000",
    "block_num": 5835,
    "last_irreversible_block": 5877,
    "traces": [
        {
            "receipt": {
                "receiver": "helloworld11",
                "act_digest": "e1b8c3433851c497fdf644a27e4d028887bcdf520685ce5d024da01a33c4650e",
                "global_sequence": 5960,
                "recv_sequence": 13,
                "auth_sequence": [
                    [
                        "helloworld11",
                        25
                    ]
                ],
                "code_sequence": 6,
                "abi_sequence": 6
            },
     

In [69]:
import time
t1 = 1560240792000000
t2 = 1560240802000000

local_time = time.localtime(t1/1e6)
r = time.strftime('%a, %d %b %Y %H:%M:%S GMT', local_time)
print(r)

local_time = time.localtime(t2/1e6)
r = time.strftime('%a, %d %b %Y %H:%M:%S GMT', local_time)
print(r)

Tue, 11 Jun 2019 16:13:12 GMT
Tue, 11 Jun 2019 16:13:22 GMT


# 取消deferred transaction

In [84]:
name = 'helloworld11'
publish_contract(name, code, abi)
r = eosapi.push_action(name, 'canceldefer', b'hello,world', {name:'active'})
print(r['processed']['action_traces'][0]['console'])


0

